In [1]:
##First we'll import all our tools

import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

/home/corbosiny/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [28]:
##Let's get our scripts ready to go, we'll read in all the 
def getData(csvName= 'transcripts.csv'):
    dataFrame = pd.read_csv(csvName)
    allScripts = dataFrame['transcript'].tolist()
    allScripts = [script.lower() for script in allScripts]
    return allScripts


useAll = False #Set to true to use all transcripts in your training data, false uses the first transcript

allScripts = getData()
transcript = ''

if not useAll:
    transcript = allScripts[0]
else:
    transcript = "\n".join(allScripts) #Joining our transcripts seperated by new lines

print(len(transcript))

17409


In [29]:
##Let's look at a list of all unique characters in our scripts, 
##we'll eventually need to one hot encode them to make training easier:
uniqueChars = sorted(list(set(transcript)))
numUniqueChars = len(uniqueChars)
print(uniqueChars)

[' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '5', '6', '9', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—']


In [30]:
##Lets make a mapping of each character to a specific number, this will help our training since we need numerical data:
charsToInt = dict((char, i) for i, char in enumerate(uniqueChars))
print(charsToInt)

{'l': 30, '5': 13, 'b': 20, 'i': 27, '?': 18, 'c': 21, ',': 6, '(': 4, 's': 37, 'v': 40, 'a': 19, '!': 1, 'h': 26, '—': 45, ' ': 0, 'm': 31, '-': 7, '3': 12, 'w': 41, '2': 11, 'o': 33, '"': 2, 'd': 22, 'p': 34, 'k': 29, 'u': 39, ')': 5, 'x': 42, '6': 14, 'z': 44, 'y': 43, '1': 10, 'g': 25, 'f': 24, 'q': 35, 'j': 28, 't': 38, 'n': 32, '.': 8, 'e': 23, ';': 17, ':': 16, '9': 15, '0': 9, 'r': 36, "'": 3}


In [31]:
##For our network we are going to train it by feeding it strings of characters and have it predict what the next
##character in the sequence will be. So to generate these sequences we will copy 100 characters from our transcript
##into a sequence, take the next character as the target answer, and then shift our window by one character and do that
##over and over again. 

lengthOfSequence = 100
def prepSequences(rawText, encoding, sequenceLength = 100): 
    data = []
    targets = []
    for i in range(0, len(rawText) - sequenceLength, 1):
        sequence = rawText[i: i+sequenceLength]
        target = rawText[i + sequenceLength]
        data.append([encoding[char] for char in sequence]) #Here we are encoding the characters to their previous assigned values
        targets.append(encoding[target])                   #Same with the target answer

    return data, targets

data, targets = prepSequences(transcript, charsToInt, lengthOfSequence)
print(data[0])

[25, 33, 33, 22, 0, 31, 33, 36, 32, 27, 32, 25, 8, 0, 26, 33, 41, 0, 19, 36, 23, 0, 43, 33, 39, 18, 4, 30, 19, 39, 25, 26, 38, 23, 36, 5, 27, 38, 3, 37, 0, 20, 23, 23, 32, 0, 25, 36, 23, 19, 38, 6, 0, 26, 19, 37, 32, 3, 38, 0, 27, 38, 18, 0, 27, 3, 40, 23, 0, 20, 23, 23, 32, 0, 20, 30, 33, 41, 32, 0, 19, 41, 19, 43, 0, 20, 43, 0, 38, 26, 23, 0, 41, 26, 33, 30, 23, 0, 38, 26]


In [32]:
##To finish off prepping our data, we need to convert x to be [samples, time steps, features]
##and we need to convert our training answers to a one hot encoding
def prepX(data, lengthOfSequence, numUniqueChars):
    data = np.reshape(data, (len(data), lengthOfSequence, 1))
    data = data / float(numUniqueChars)
    return data

def prepY(targets):
    targets = np_utils.to_categorical(targets)
    return targets

preppedX = prepX(data, lengthOfSequence, numUniqueChars)
preppedY = prepY(targets)


##The last thing we can do before we train is get our model set up
def generateModel(X, y):
    model = Sequential()
    model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(256))
    model.add(Dropout(0.2))
    model.add(Dense(y.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

model = generateModel(preppedX, preppedY)

In [27]:
##Training time!

def trainModel(model, X, y, numEpochs= 20, batchSize= 128):
    filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5" #replace with lowest loss file
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
    callbacks_list = [checkpoint]

    model.fit(X, y, epochs = numEpochs, batch_size= batchSize, callbacks=callbacks_list)
    return model

model = trainModel(model, preppedX, preppedY, 1)

Epoch 1/1
3041/3041 [==============================] - 7s 2ms/step - loss: 7.1352

Epoch 00001: loss improved from inf to 7.13517, saving model to weights-improvement-01-7.1352.hdf5


In [33]:
##The code below loads back in the best weights we add
def loadModel(model, filename):  #replace with best weights file for your training
    model.load_weights(filename)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

model = loadModel(model, "weights-improvement-01-3.0578.hdf5")

In [48]:
##Now for text generation
def generateSeedFromData(data):
    start = np.random.randint(0, len(data)-1)
    pattern = data[start]
    return pattern



def generateText(model, pattern, decoding, length= 1000, vocabSize= 47, delimeter= ''):
    text= delimeter.join([decoding[value] for value in pattern])
    for i in range(length):
        preppedPattern = prepPattern(pattern, vocabSize)
        prediction = model.predict(preppedPattern, verbose= 0)
        index = np.argmax(prediction)
        result = decoding[index]
        text += delimeter + result
        pattern.append(index)
        pattern = pattern[1:]
        
    return text

def prepPattern(pattern, vocabSize):
    pattern = np.reshape(pattern, (1, len(pattern), 1))
    pattern = pattern / float(vocabSize)
    return pattern

intToChar = dict((i, char) for i, char in enumerate(uniqueChars))  #creating a demapping of our original encoding
seed = generateSeedFromData(data) #get a random starting point from our paper and let the network continue the writing
print("Starting Seed: ", ''.join([intToChar[value] for value in seed]), end= '\n\n\n')
numCharacters= 200   #length of each window the network will use to predict the output
text = generateText(model, seed, intToChar, length= numCharacters, vocabSize= numUniqueChars)
print(text)

KeyError: 1278

In [35]:
##Let's look at a list of all unique characters in our scripts, 
##we'll eventually need to one hot encode them to make training easier:
uniqueWords = sorted(transcript.split(' '))
numUniqueWords = len(uniqueWords)

##Lets make a mapping of each character to a specific number, this will help our training since we need numerical data:
stringToInt = dict((string, i) for i, string in enumerate(uniqueWords))

In [60]:
lengthOfSequence = 25
words = transcript.split(" ")
data, targets = prepSequences(words, stringToInt, lengthOfSequence)
preppedX = prepX(data, lengthOfSequence, numUniqueWords)
preppedY = prepY(targets)
print([intToString[point] for point in data[10]], intToString[targets[10]])

['been', 'blown', 'away', 'by', 'the', 'whole', 'thing.', 'in', 'fact,', "i'm", 'leaving.(laughter)there', 'have', 'been', 'three', 'themes', 'running', 'through', 'the', 'conference', 'which', 'are', 'relevant', 'to', 'what', 'i'] want


In [52]:
model = generateModel(preppedX, preppedY)
model = trainModel(model, preppedX, preppedY, 5, 5)

Epoch 1/5
3041/3041 [==============================] - 34s 11ms/step - loss: 7.2040

Epoch 00001: loss improved from inf to 7.20399, saving model to weights-improvement-01-7.2040.hdf5
Epoch 2/5
3041/3041 [==============================] - 32s 10ms/step - loss: 6.6807

Epoch 00002: loss improved from 7.20399 to 6.68073, saving model to weights-improvement-02-6.6807.hdf5
Epoch 3/5
3041/3041 [==============================] - 32s 11ms/step - loss: 6.5085

Epoch 00003: loss improved from 6.68073 to 6.50845, saving model to weights-improvement-03-6.5085.hdf5
Epoch 4/5
3041/3041 [==============================] - 32s 11ms/step - loss: 6.3964

Epoch 00004: loss improved from 6.50845 to 6.39644, saving model to weights-improvement-04-6.3964.hdf5
Epoch 5/5
3041/3041 [==============================] - 32s 11ms/step - loss: 6.3664

Epoch 00005: loss improved from 6.39644 to 6.36638, saving model to weights-improvement-05-6.3664.hdf5


In [53]:

model = loadModel(model, "weights-improvement-05-6.3664.hdf5")
intToString = dict((i, word) for i, word in enumerate(uniqueWords))  #creating a demapping of our original encoding
seed = generateSeedFromData(data) #get a random starting point from our paper and let the network continue the writing
numWords= 100   #length of each window the network will use to predict the output
text = generateText(model, seed, intToString, length= numWords, vocabSize= numUniqueWords, delimeter= ' ')
print(text)

conference which are relevant to what i want to talk about. one is the extraordinary evidence of human creativity in all of the presentations that the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
